<div style="width:100%;text-align: center;">
    <img src="https://user.oc-static.com/upload/2019/02/24/15510245026714_Seattle_logo_landscape_blue-black.png" />
</div>

# Introduction

Vous travaillez pour la ville de Seattle. Pour atteindre son objectif de ville neutre en émissions de carbone en 2050, votre équipe s’intéresse de près aux émissions des bâtiments non destinés à l’habitation.

Des relevés minutieux ont été effectués par vos agents en 2015 et en 2016. Cependant, ces relevés sont coûteux à obtenir, et à partir de ceux déjà réalisés, vous voulez tenter de prédire les émissions de CO2 et la consommation totale d’énergie de bâtiments pour lesquels elles n’ont pas encore été mesurées.

Votre prédiction se basera sur les données déclaratives du permis d'exploitation commerciale (taille et usage des bâtiments, mention de travaux récents, date de construction..)

Vous cherchez également à évaluer l’intérêt de l’"ENERGY STAR Score" pour la prédiction d’émissions, qui est fastidieux à calculer avec l’approche utilisée actuellement par votre équipe.


In [ ]:
# import des bibliothèques utilisées
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

**postulat:** peu d'immeuble très haut entraineront une consommation d'énergie importante, ce qui pourrait passer pour des outliers seraient donc des valeurs cohérentes. Ils serait intéressant de ramener la consommation électrique proportionnellement à la surface du bâtiment, nous pourrions ainsi remarquer de vrai outliers qui ont une consommation anormale.

In [ ]:
# chargement des données 
data = pd.read_csv("/kaggle/input/refined-data/data_hard_refined.csv")
data_full = pd.read_csv("/kaggle/input/refined-data/data_filtered.csv")
objectColumns = list(data.dtypes[data.dtypes == np.object].index)
numericColumns = list(data.dtypes[data.dtypes != np.object].index)
objectColumns = list(data.dtypes[data.dtypes == np.object].index)
numericColumns = list(data.dtypes[data.dtypes != np.object].index)
print(objectColumns)
print(numericColumns)

In [ ]:
print(data['BuildingType'].unique())
print(data['PrimaryPropertyType'].unique())

In [ ]:
# analyse de variables quantitatives
data['Energy/Surface'] = data['SiteEnergyUse(kBtu)']/data['PropertyGFATotal']
data['GHG/Surface'] = data['TotalGHGEmissions']/data['PropertyGFATotal']
columns_to_boxplot=['SiteEnergyUse(kBtu)','Energy/Surface', 'TotalGHGEmissions', 'GHG/Surface']
for column in columns_to_boxplot:
    print('column: {}'.format(column))
    n_bins = 12
    subset = data[column]
    print('all datas: \n\tmean: {}\n\tmed: {}\n\tmod: {}\n\tvar: {}\n\tstd: {}\n\tskew: {}\n\tkur: {}'
          .format(subset.mean(),subset.median(), subset.mode()[0],subset.var(ddof=0),subset.std(ddof=0),subset.skew(),subset.kurtosis()))
    plt.hist(subset,n_bins,label='all datas', alpha=0.5)
    plt.title(column)
    plt.legend(loc='best')
    plt.show()
    subset.plot.box(vert=False)
    plt.title(column)
    plt.show()
    # https://towardsdatascience.com/create-and-customize-boxplots-with-pythons-matplotlib-to-get-lots-of-insights-from-your-data-d561c9883643 
    print('='*120)

In [ ]:
# détection des outliers 
for column in columns_to_boxplot:
    i= 0
    quartile_1 = data[column].quantile(0.25)
    quartile_3 = data[column].quantile(0.75)   
    iqr = quartile_3 - quartile_1
    min_value = quartile_1 - 1.5*iqr
    max_value = quartile_3 + 1.5*iqr
    outliers = data[(data[column]>max_value) | (data[column]<min_value)]
    i += outliers.shape[0]
    print('colonnes {}, {} outliers détectés'.format(column,i))

Ramener notre analyse à la surface réduit bien notre quantité d'outliers, comme je l'avais pensé. Essayons de faire la même chose par catégorie d'habitations car il parait évident qu'un hopital consommera plus qu'un hotel.

In [ ]:
# analyse des variables quantitatives continues: pour les groupes de PrimaryPropertyType)
groups = data['PrimaryPropertyType'].unique()

for column in columns_to_boxplot:
    print('column: {}'.format(column))
    n_bins = 12
    values=[]
    assessment =  []
    subset = data[column]
    # var = variance empirique, std = ecart type empirique
    print('all datas: \n\tmean: {}\n\tmed: {}\n\tmod: {}\n\tvar: {}\n\tstd: {}\n\tskew: {}\n\tkur: {}'
          .format(subset.mean(),subset.median(), subset.mode()[0],subset.var(ddof=0),subset.std(ddof=0),subset.skew(),subset.kurtosis()))
    plt.hist(subset,n_bins,label='all datas', alpha=0.5)
    plt.show()
    subset.plot.box(vert=False)
    plt.title(column)
    plt.show()
    
    fig = plt.figure(figsize=(36,18))
    for group in  groups:
        subset = data[data.PrimaryPropertyType == group][column] # Création du sous-échantillon
        values.append(subset)
        assessment.append([group,subset.mean(),subset.median(), subset.mode()[0],subset.var(ddof=0),subset.std(ddof=0),subset.skew(),subset.kurtosis()])
        plt.hist(subset,n_bins,label=group, alpha=0.3) 
        
    plt.legend(loc='best')
    plt.title(column)
    plt.show() # Affiche l'histogramme
    fig = plt.figure(figsize=(36,18))
    plt.hist(values, n_bins, histtype='bar', label=groups)
    plt.legend(loc='best')
    plt.title(column)
    plt.show() # Affiche l'histogramme
    
    df_assessment = pd.DataFrame(assessment,columns=['group', 'moy', 'med', 'mod','var','std','skew','kur'])
    display(df_assessment)
            
    # https://towardsdatascience.com/create-and-customize-boxplots-with-pythons-matplotlib-to-get-lots-of-insights-from-your-data-d561c9883643 
    
    fig, ax = plt.subplots(figsize=(36,10))
    ax.boxplot(values, labels=groups)
    plt.show()
    
    print('='*120)

In [ ]:
#### détection des outliers par groupe
data_refined = data.copy()
columns_outliers_to_delete=['Energy/Surface', 'GHG/Surface']
for column in columns_outliers_to_delete:
    print('colonne {}'.format(column))
    i= 0
    for group in groups:
        j=0
        subset = data_refined[data_refined.PrimaryPropertyType == group]
        quartile_1 = subset[column].quantile(0.25)
        quartile_3 = subset[column].quantile(0.75)   
        iqr = quartile_3 - quartile_1
        min_value = quartile_1 - 1.5*iqr
        max_value = quartile_3 + 1.5*iqr
        outliers = subset[(subset[column]>max_value) | (subset[column]<min_value)]
        j += outliers.shape[0]
        i += j 
        print('groupe {}, {} outliers détectés'.format(group,j))
        data_refined.drop(outliers.index.values,inplace=True)
    print('Total {}, {} outliers détectés'.format(column,i))
    print('='*120)

In [ ]:
print(data.shape)
print(data_refined.shape)
data_refined.to_csv('data_without_outliers.csv', index=False)

In [ ]:
# détection des outliers 
data_refine = data.copy()
columns_outliers_to_delete==['SiteEnergyUse(kBtu)', 'TotalGHGEmissions']
for column in columns_outliers_to_delete:
    i= 0
    quartile_1 = data_refine[column].quantile(0.25)
    quartile_3 = data_refine[column].quantile(0.75)   
    iqr = quartile_3 - quartile_1
    min_value = quartile_1 - 1.5*iqr
    max_value = quartile_3 + 1.5*iqr
    outliers = data_refine[(data_refine[column]>max_value) | (data_refine[column]<min_value)]
    i += outliers.shape[0]
    print('colonnes {}, {} outliers détectés'.format(column,i))
    data_refine.drop(outliers.index.values,inplace=True)

In [ ]:
print(data.shape)
print(data_refine.shape)

On peut observer qu'en procédant par force brute nous aurions détectés et supprimés 50% d'outliers en plus.